<a href="https://colab.research.google.com/github/Cole-Pd/DeepDive/blob/main/Project_5_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0 Build the Pickles

In [1]:
%%capture
!python -m textblob.download_corpora

In [2]:
import pandas as pd
from textblob import TextBlob, Word
import pickle
url = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv'
df = pd.read_csv(url)

In [3]:
df['tb'] = df['text'].apply(TextBlob)
l = df['tb'][:20000].apply(lambda x: [x.singularize() for x in x.words])
pickle.dump(l, open('pickle_part_1.p','wb'))
%reset -f
import pickle
l = pickle.load(open('pickle_part_1.p','rb'))

In [4]:
%%capture
!python -m textblob.download_corpora

In [5]:
import pandas as pd
from textblob import TextBlob, Word
import pickle
url = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv'
df = pd.read_csv(url)

In [6]:
df['tb'] = df['text'].apply(TextBlob)
a = df['tb'][20000:].apply(lambda x: [x.singularize() for x in x.words])
pickle.dump(a, open('pickle_part_2.p','wb'))
%reset -f
import pickle
a = pickle.load(open('pickle_part_2.p','rb'))

In [7]:
%reset -f

# Part 1: Find a Random Page and It's 10 Nearest Neighbors.

In [8]:
import pandas as pd
import numpy as np
import pickle #yay :)
import random
from textblob import TextBlob, Word
import nltk
nltk.download('omw-1.4')

from sklearn.feature_extraction.text import CountVectorizer as BagOfWords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [9]:
%%capture
!python -m textblob.download_corpora

In [10]:
url = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv'
df = pd.read_csv(url)

In [11]:
df = df.drop(29122, axis = 0).reset_index(drop = True)

In [12]:
#importing our pickle file was good practice.
#however it resulted in a file that for some reason was blowing up our ram
#to get around this we split our strings into two pickles
#they are below and squished together

p1 = pickle.load(open('/content/pickle_part_1.p','rb'))
p2 = pickle.load(open('/content/pickle_part_2.p','rb'))

In [13]:
singularized = pd.concat([p1,p2], axis = 0)
singularized = singularized.drop(29122).reset_index(drop = True)
singularized

0        [digby, morrell, born, 10, october, 1979, is, ...
1        [alfred, j, lewy, aka, sandy, lewy, graduated,...
2        [harpdog, brown, is, a, singer, and, harmonica...
3        [franz, rottensteiner, born, in, waidmannsfeld...
4        [henry, krvit, born, 30, december, 1974, in, t...
                               ...                        
42780    [motoaki, takenouchi, born, july, 8, 1967, sai...
42781    [alan, graham, judge, born, 14, may, 1960, is,...
42782    [eduardo, lara, lozano, born, 4, september, 19...
42783    [tatiana, faberg, is, an, author, and, faberg,...
42784    [kenneth, thoma, born, february, 24, 1938, wa,...
Name: tb, Length: 42785, dtype: object

In [14]:
Bag_of_Words = BagOfWords(stop_words = 'english')
word_vector = Bag_of_Words.fit_transform(singularized.str.join(', '))

In [15]:
tf_idf = TfidfTransformer()
tf_idf_vw = tf_idf.fit_transform(word_vector)

In [16]:
tf_idf_vw.shape #Big Matrix

(42785, 404863)

In [17]:
nearest_neighbors = NearestNeighbors().fit(tf_idf_vw)

In [18]:
random.seed(42)
Page = random.randrange(0, len(df))
Wikipage = tf_idf_vw[Page]
Wikipage

<1x404863 sparse matrix of type '<class 'numpy.float64'>'
	with 78 stored elements in Compressed Sparse Row format>

In [19]:
distances, indecies = nearest_neighbors.kneighbors(
    X = Wikipage,
    n_neighbors = 11
)

In [20]:
indecies

array([[41905, 36112, 20856, 36166, 26386, 25545,  8366,   895, 12286,
         9732, 31511]])

In [21]:
nn_index = list(indecies[0][0:])

In [22]:
nn_index_dist = list(distances[0][0:])

In [23]:
df.iloc[41905]

URI               <http://dbpedia.org/resource/Tom_Jaine>
name                                            Tom Jaine
text    tom jaine born 4 june 1943 is a former restaur...
Name: 41905, dtype: object

In [24]:
ten_nn = df.iloc[[41905, 36112, 20856, 36166, 26386, 25545,  8366,   895, 12286,
         9732, 31511]]
ten_nn

,URI,name,text
41905,<http://dbpedia.org/resource/Tom_Jaine>,Tom Jaine,tom jaine born 4 june 1943 is a former restaur...
36112,<http://dbpedia.org/resource/Joanna_Blythman>,Joanna Blythman,joanna blythman is a british investigative foo...
20856,<http://dbpedia.org/resource/Matthew_Fort>,Matthew Fort,matthew fort born 29 january 1947 is a british...
36166,<http://dbpedia.org/resource/Stefan_Gates>,Stefan Gates,stefan gates born 19 september 1967 in london ...
26386,<http://dbpedia.org/resource/Oran_B._Hesterman>,Oran B. Hesterman,oran b hesterman is the president and chief ex...
25545,<http://dbpedia.org/resource/Russ_Parsons>,Russ Parsons,russ parsons is the food editor and columnist ...
8366,<http://dbpedia.org/resource/Charles_Campion>,Charles Campion,charles campion is an english food critic who ...
895,<http://dbpedia.org/resource/Manju_Malhi>,Manju Malhi,manju malhi is a britishborn chef and food wri...
12286,<http://dbpedia.org/resource/Anna_Lapp%C3%A9>,Anna Lapp%C3%A9,anna lapp is an author and educator known for ...
9732,<http://dbpedia.org/resource/Rose_Prince_(writ...,Rose Prince (writer),rose prince born 4 december 1962 in hampshire ...


In [25]:
sentiments = []
for i in range(1,11):
  sentiments.append(TextBlob(ten_nn['text'].iloc[i]).sentiment)
sentiments

[Sentiment(polarity=0.21978021978021978, subjectivity=0.3676739926739927),
 Sentiment(polarity=0.11319444444444444, subjectivity=0.28402777777777777),
 Sentiment(polarity=0.15137406453195923, subjectivity=0.5343454790823213),
 Sentiment(polarity=0.2042748917748918, subjectivity=0.4016504329004329),
 Sentiment(polarity=0.27166666666666667, subjectivity=0.245),
 Sentiment(polarity=0.1388888888888889, subjectivity=0.28144078144078144),
 Sentiment(polarity=0.1300407925407925, subjectivity=0.34867216117216115),
 Sentiment(polarity=0.12348484848484849, subjectivity=0.36604683195592286),
 Sentiment(polarity=0.1010942760942761, subjectivity=0.29131054131054135),
 Sentiment(polarity=0.05574009324009324, subjectivity=0.20645956607495072)]

# Part 2: Of the 10 Nearest Neighbors, Find their Whole Wiki Page and Order Their Indecies by the Whole Page.

In [26]:
%%capture
!pip3 install wikipedia-api

In [27]:
import wikipediaapi
wikipedia = wikipediaapi.Wikipedia(user_agent = 'agoobi')

In [28]:
Person = 'Tom Jaine'
Tom_Jaine = wikipedia.page(Person).text
Tom_Jaine

"Tom Jaine (born 4 June 1943) is a former restaurateur, a food writer and until recently the publisher of Prospect Books.\nHe was educated at Kingswood School (1955–1959) and at Balliol College, Oxford where he studied Modern history (1961–1964). He worked as an archivist from 1964 to 1973 and a restaurateur from 1974 to 1984. From 1984 to 1988, he organised the Oxford Symposium on Food and Cookery, and from 1989 to 1994 he waso editor of the annual Good Food Guide. From 1993 to 2016 he was the proprietor of Prospect Books, a prize-winning publishing company specialising in food and food history.\nHe is the author of four books and has written for  The Times, The Guardian, The Sunday Times, The Sunday Telegraph, The Evening Standard and many other newspapers and magazines. He has presented The Food Programme and appeared on it many times, has done interviews for the BBC, BBC TV, and ITV, and a series of programmes about food and cookery in the Balkans for BBC Radio 4.\nHe was Glenfiddi

In [29]:
(Tom_Jaine
.replace("\n"," ")
.replace("\'s",'')
.replace('\'','')
.replace("(", "")
.replace(")", "")
.replace('"', "")
)

'Tom Jaine born 4 June 1943 is a former restaurateur, a food writer and until recently the publisher of Prospect Books. He was educated at Kingswood School 1955–1959 and at Balliol College, Oxford where he studied Modern history 1961–1964. He worked as an archivist from 1964 to 1973 and a restaurateur from 1974 to 1984. From 1984 to 1988, he organised the Oxford Symposium on Food and Cookery, and from 1989 to 1994 he waso editor of the annual Good Food Guide. From 1993 to 2016 he was the proprietor of Prospect Books, a prize-winning publishing company specialising in food and food history. He is the author of four books and has written for  The Times, The Guardian, The Sunday Times, The Sunday Telegraph, The Evening Standard and many other newspapers and magazines. He has presented The Food Programme and appeared on it many times, has done interviews for the BBC, BBC TV, and ITV, and a series of programmes about food and cookery in the Balkans for BBC Radio 4. He was Glenfiddich Restau

In [30]:
Tom_Jaine = TextBlob(Tom_Jaine)
Tom_Jaine.sentiment

Sentiment(polarity=0.23452380952380952, subjectivity=0.2777777777777778)

In [31]:
ten_nn = df.iloc[[41905, 36112, 20856, 36166, 26386, 25545,  8366,   895, 12286,
         9732, 31511]]

In [32]:
wiki_nn = []

for i in range(11):
  nn = wikipedia.page(ten_nn['name'].iloc[i]).text
  nn = (nn
.replace("\n"," ")
.replace("\'s",'')
.replace('\'','')
.replace("(", "")
.replace(")", "")
.replace('"', "")
  )

  wiki_nn.append(nn)

In [33]:
print(wiki_nn[4])

Oran B. Hesterman is the president and chief executive officer of Fair Food Network, a non-profit organization based in Ann Arbor Michigan, is a national leader in sustainable agriculture and food systems and the author of Fair Food: Growing a Healthy, Sustainable Food System for All Public Affairs, as well as more than 400 reports and articles  on subjects such as cover crops, crop rotation, and the impact of philanthropic investments on food systems practice and policy. Since its release, Fair Food has garnered a significant amount of attention, with over 4,000 food and social justice activists attending book events nationwide in 2011.  Beyond listing the health, environment, and economic dysfunctions of the current broken American food system, the book presents burgeoning success stories and illuminates a clear path toward a more sustainable and equitable food future.   The New York Times calls it “an important, accessible book on a crucial subject.” Before starting Fair Food Networ

In [ ]:
#Anna Lapp and Bill Glasson apparently do not have a wikipedia page?

sentiments = []

for page in wiki_nn:
  sent = TextBlob(page).sentiment
  sentiments.append(sent)

sentiments

In [35]:
Bag_of_Words = BagOfWords(stop_words = 'english')
word_vector = Bag_of_Words.fit_transform(wiki_nn)
tf_idf = TfidfTransformer()
tf_idf_vw = tf_idf.fit_transform(word_vector)
nearest_neighbors = NearestNeighbors().fit(tf_idf_vw)
distances, position = nearest_neighbors.kneighbors(
    X = tf_idf_vw[0],
    n_neighbors = 11)

In [36]:
# Note these are the indecies and python starts counting at zero.
ranked = list(position[0][1:])
ranked

[8, 1, 3, 5, 10, 2, 6, 9, 7, 4]

In [37]:
df['name']

0                             Digby Morrell
1                            Alfred J. Lewy
2                             Harpdog Brown
3                       Franz Rottensteiner
4                                    G-Enka
                        ...                
42780                    Motoaki Takenouchi
42781    Alan Judge (footballer, born 1960)
42782                          Eduardo Lara
42783                  Tatiana Faberg%C3%A9
42784                        Kenneth Thomas
Name: name, Length: 42785, dtype: object

# Part 3: Make Interactive

In [123]:
word_vector = Bag_of_Words.fit_transform(singularized.str.join(', '))
tf_idf = TfidfTransformer()
tf_idf_vw = tf_idf.fit_transform(word_vector)
nearest_neighbors = NearestNeighbors().fit(tf_idf_vw)

In [137]:


def wiki_search():
  '''Propmpts User Imput to Search for 10 NN. '''
  search = input('')

  if (search == df['name']).any():

    distances, people = nearest_neighbors.kneighbors(
    X = tf_idf_vw[int(df[df['name'] == search].index[0])],
    n_neighbors = 11)

    nn_df = df.iloc[list(people[0][:])]
    sent = []
    wiki_text = []

    for person in list(people[0][:]):
      wiki_page = wikipedia.page(df['name'].iloc[person]).text
      wiki_page_clean = (
               wiki_page
      .replace("\n"," ")
      .replace("\'s",'')
      .replace('\'','')
      .replace("(", ""  )
      .replace(")", "")
      .replace('"', "")
       )

      wiki_text.append(wiki_page_clean)
      sent.append(TextBlob(wiki_page_clean).sentiment)

    nn_df.loc[:, 'Sentiment'] = sent
    nn_df.loc[:, 'Wiki Text'] = wiki_text

    return nn_df


  else:
      print("We Couldn't Find Them Please Try Again! :( ")

In [ ]:
df['name']

In [ ]:
wiki_search() #Digby Morrell